# Feature Engineering e Análise - Hackathon Forecast 2025

Este notebook implementa e analisa as features para o modelo de previsão de vendas, incluindo features temporais, de lag, estatísticas móveis e seleção de features.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Importar módulos do projeto
import sys
sys.path.append('../src')
from data.ingestion import DataIngestion
from data.preprocessing import DataPreprocessor
from features.engineering import FeatureEngineer
from features.selection import FeatureSelector

## 1. Carregamento e Preparação dos Dados

In [ ]:
# Carregar e processar dados
ingestion = DataIngestion()
preprocessor = DataPreprocessor()
feature_engineer = FeatureEngineer()
feature_selector = FeatureSelector()

print("Carregando dados...")
transactions_df = ingestion.load_transactions('../hackathon_2025_templates/')
products_df = ingestion.load_products('../hackathon_2025_templates/')
stores_df = ingestion.load_stores('../hackathon_2025_templates/')

print("Processando dados...")
transactions_clean = preprocessor.clean_transactions(transactions_df)
weekly_sales = preprocessor.aggregate_weekly_sales(transactions_clean)
merged_data = preprocessor.merge_master_data(weekly_sales, products_df, stores_df)

print(f"Dados base: {merged_data.shape}")

## 2. Engenharia de Features Temporais

In [ ]:
# Criar features temporais
print("Criando features temporais...")
temporal_features = feature_engineer.create_temporal_features(merged_data)

print(f"Features temporais criadas: {temporal_features.shape}")
print("\nNovas colunas temporais:")
temporal_cols = [col for col in temporal_features.columns if col not in merged_data.columns]
for col in temporal_cols:
    print(f"- {col}")

In [ ]:
# Visualizar distribuição das features temporais
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=('Semana do Ano', 'Mês', 'Trimestre', 
                   'Dia da Semana', 'É Início de Mês', 'É Fim de Mês')
)

# Semana do ano
fig.add_trace(
    go.Histogram(x=temporal_features['semana_ano'], name='Semana'),
    row=1, col=1
)

# Mês
fig.add_trace(
    go.Histogram(x=temporal_features['mes'], name='Mês'),
    row=1, col=2
)

# Trimestre
fig.add_trace(
    go.Histogram(x=temporal_features['trimestre'], name='Trimestre'),
    row=1, col=3
)

# Dia da semana
fig.add_trace(
    go.Histogram(x=temporal_features['dia_semana'], name='Dia Semana'),
    row=2, col=1
)

# Início de mês
fig.add_trace(
    go.Histogram(x=temporal_features['inicio_mes'], name='Início Mês'),
    row=2, col=2
)

# Fim de mês
fig.add_trace(
    go.Histogram(x=temporal_features['fim_mes'], name='Fim Mês'),
    row=2, col=3
)

fig.update_layout(height=600, title_text="Distribuição das Features Temporais", showlegend=False)
fig.show()

## 3. Features de Produto e PDV

In [ ]:
# Criar features de produto
print("Criando features de produto...")
product_features = feature_engineer.create_product_features(temporal_features)

print(f"Features de produto: {product_features.shape}")
product_cols = [col for col in product_features.columns if col not in temporal_features.columns]
print("\nNovas features de produto:")
for col in product_cols:
    print(f"- {col}")

In [ ]:
# Criar features de PDV
print("Criando features de PDV...")
store_features = feature_engineer.create_store_features(product_features)

print(f"Features de PDV: {store_features.shape}")
store_cols = [col for col in store_features.columns if col not in product_features.columns]
print("\nNovas features de PDV:")
for col in store_cols:
    print(f"- {col}")

## 4. Features de Lag e Estatísticas Móveis

In [ ]:
# Criar features de lag
print("Criando features de lag...")
lag_features = feature_engineer.create_lag_features(store_features)

print(f"Features com lag: {lag_features.shape}")
lag_cols = [col for col in lag_features.columns if 'lag_' in col or 'ma_' in col or 'std_' in col]
print(f"\nFeatures de lag criadas ({len(lag_cols)}):")
for col in lag_cols[:10]:  # Mostrar apenas as primeiras 10
    print(f"- {col}")
if len(lag_cols) > 10:
    print(f"... e mais {len(lag_cols) - 10} features")

In [ ]:
# Visualizar correlação entre lags
lag_correlation = lag_features[lag_cols].corr()

fig = px.imshow(
    lag_correlation,
    title="Correlação entre Features de Lag",
    aspect="auto"
)
fig.show()

## 5. Análise de Importância das Features

In [ ]:
# Preparar dados para análise de importância
# Remover linhas com NaN (devido aos lags)
analysis_data = lag_features.dropna()

# Separar features numéricas
numeric_features = analysis_data.select_dtypes(include=[np.number]).columns.tolist()
if 'quantidade' in numeric_features:
    numeric_features.remove('quantidade')

X = analysis_data[numeric_features]
y = analysis_data['quantidade']

print(f"Dados para análise: {X.shape}")
print(f"Features numéricas: {len(numeric_features)}")

In [ ]:
# Calcular importância com Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X, y)

# Criar DataFrame com importâncias
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

# Visualizar top 20 features mais importantes
top_features = feature_importance.head(20)

fig = px.bar(
    top_features,
    x='importance',
    y='feature',
    orientation='h',
    title="Top 20 Features Mais Importantes (Random Forest)"
)
fig.update_layout(height=600)
fig.show()

## 6. Seleção de Features

In [ ]:
# Aplicar diferentes métodos de seleção
print("Aplicando seleção de features...")

# Método 1: SelectKBest
selector_kbest = SelectKBest(score_func=f_regression, k=50)
X_kbest = selector_kbest.fit_transform(X, y)
selected_features_kbest = X.columns[selector_kbest.get_support()]

print(f"SelectKBest: {len(selected_features_kbest)} features selecionadas")

# Método 2: RFE com Random Forest
rfe = RFE(estimator=RandomForestRegressor(n_estimators=50, random_state=42), n_features_to_select=50)
X_rfe = rfe.fit_transform(X, y)
selected_features_rfe = X.columns[rfe.support_]

print(f"RFE: {len(selected_features_rfe)} features selecionadas")

# Método 3: Baseado em importância do Random Forest
threshold = feature_importance['importance'].quantile(0.8)  # Top 20%
selected_features_rf = feature_importance[feature_importance['importance'] >= threshold]['feature'].tolist()

print(f"RF Importance: {len(selected_features_rf)} features selecionadas")

In [ ]:
# Comparar métodos de seleção
methods_comparison = pd.DataFrame({
    'Método': ['SelectKBest', 'RFE', 'RF Importance'],
    'Num Features': [len(selected_features_kbest), len(selected_features_rfe), len(selected_features_rf)]
})

fig = px.bar(
    methods_comparison,
    x='Método',
    y='Num Features',
    title="Comparação dos Métodos de Seleção de Features"
)
fig.show()

# Encontrar features comuns
common_features = set(selected_features_kbest) & set(selected_features_rfe) & set(selected_features_rf)
print(f"\nFeatures selecionadas por todos os métodos: {len(common_features)}")
print("Features comuns:")
for feature in sorted(common_features):
    print(f"- {feature}")

## 7. Análise de Correlação das Features Selecionadas

In [ ]:
# Analisar correlação das features mais importantes
top_30_features = feature_importance.head(30)['feature'].tolist()
correlation_matrix = X[top_30_features].corr()

fig = px.imshow(
    correlation_matrix,
    title="Matriz de Correlação - Top 30 Features",
    aspect="auto"
)
fig.show()

In [ ]:
# Identificar features altamente correlacionadas
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_value = correlation_matrix.iloc[i, j]
        if abs(corr_value) > 0.8:  # Correlação alta
            high_corr_pairs.append({
                'Feature 1': correlation_matrix.columns[i],
                'Feature 2': correlation_matrix.columns[j],
                'Correlação': corr_value
            })

if high_corr_pairs:
    high_corr_df = pd.DataFrame(high_corr_pairs).sort_values('Correlação', key=abs, ascending=False)
    print("Features com alta correlação (>0.8):")
    print(high_corr_df)
else:
    print("Nenhuma correlação alta encontrada entre as top features.")

## 8. Análise de Distribuição das Features

In [ ]:
# Analisar distribuição das top features
top_10_features = feature_importance.head(10)['feature'].tolist()

fig = make_subplots(
    rows=2, cols=5,
    subplot_titles=top_10_features
)

for i, feature in enumerate(top_10_features):
    row = (i // 5) + 1
    col = (i % 5) + 1
    
    fig.add_trace(
        go.Histogram(x=X[feature], name=feature, showlegend=False),
        row=row, col=col
    )

fig.update_layout(height=600, title_text="Distribuição das Top 10 Features")
fig.show()

## 9. Feature Engineering Avançado

In [ ]:
# Criar features de interação
print("Criando features de interação...")

# Interações entre top features
interaction_features = analysis_data.copy()

# Exemplo: interação entre categoria e tipo de PDV
if 'categoria_produto_encoded' in interaction_features.columns and 'tipo_pdv_encoded' in interaction_features.columns:
    interaction_features['categoria_x_tipo_pdv'] = (
        interaction_features['categoria_produto_encoded'] * interaction_features['tipo_pdv_encoded']
    )

# Interação entre lag e sazonalidade
if 'lag_1' in interaction_features.columns and 'semana_ano' in interaction_features.columns:
    interaction_features['lag1_x_semana'] = (
        interaction_features['lag_1'] * interaction_features['semana_ano']
    )

print(f"Features com interações: {interaction_features.shape}")

## 10. Resumo e Recomendações

In [ ]:
# Resumo final
print("=== RESUMO DA ENGENHARIA DE FEATURES ===")
print(f"\nDados originais: {merged_data.shape}")
print(f"Features finais: {lag_features.shape}")
print(f"Features numéricas: {len(numeric_features)}")

print("\n=== TOP 10 FEATURES MAIS IMPORTANTES ===")
for i, row in feature_importance.head(10).iterrows():
    print(f"{i+1:2d}. {row['feature']:<25} ({row['importance']:.4f})")

print("\n=== RECOMENDAÇÕES ===")
print("1. Usar features de lag como principais preditores")
print("2. Incluir features temporais para capturar sazonalidade")
print("3. Considerar features de produto e PDV para segmentação")
print("4. Aplicar seleção de features para reduzir overfitting")
print("5. Monitorar correlações altas entre features")

print("\n=== PRÓXIMOS PASSOS ===")
print("1. Testar diferentes janelas de lag")
print("2. Experimentar transformações (log, sqrt)")
print("3. Criar features de interação mais complexas")
print("4. Validar features com cross-validation temporal")

In [ ]:
# Salvar features processadas para uso nos modelos
print("Salvando features processadas...")
lag_features.to_parquet('../data/processed/features_engineered.parquet')
feature_importance.to_csv('../data/processed/feature_importance.csv', index=False)

# Salvar lista de features selecionadas
pd.Series(list(common_features)).to_csv('../data/processed/selected_features.csv', index=False, header=['feature'])

print("Features salvas com sucesso!")